In [ ]:
from pathlib import Path
import pickle
from typing import Dict, List, Optional, Tuple
from datetime import datetime

class RelationDB:
    def __init__(
        self,
        year: int,
        quarter: str,
        top_n: Optional[int] = 5,
        use_cache: bool = True,
        cache_dir: Optional[str] = None,
        force_live: bool = False
    ):
        self.year = year
        self.quarter = quarter
        self.top_n = top_n
        self.cache_dir = Path(cache_dir) if cache_dir else Path("cache")
        self.cache_file = self.cache_dir / f"relation_db_{year}_{quarter}.pkl"
        
        if use_cache and not force_live:
            if self._load_cache():
                return
                
        self._load_live_data()

    def _load_live_data(self) -> None:
        """Load live data from gsam_fe"""
        import gsam_fe
        self.d = gsam_fe.get_top_customers_info_for_companies(
            self.year, self.quarter, top_n=self.top_n, use_cache=True
        )
        self._validate_data()
        self._save_cache()

    def _validate_data(self) -> None:
        """Validate required data structures"""
        required_maps = [
            "ticker to bbg cid map",
            "bbg cid to refinitiv id map",
            "top customers final struct",
            "top suppliers final struct",
            "cons data final struct",
            "bbg cid to name map"
        ]
        missing = [k for k in required_maps if k not in self.d]
        if missing:
            raise ValueError(f"Missing required maps: {missing}")

    def _get_cache_path(self) -> Path:
        """Get and ensure cache path exists"""
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        return self.cache_file
        
    def _load_cache(self) -> bool:
        """Load cached relationship data"""
        try:
            cache_path = self._get_cache_path()
            if not cache_path.exists():
                return False
                
            with open(cache_path, 'rb') as f:
                self.d = pickle.load(f)
            return True
        except Exception as e:
            print(f"Error loading cache: {e}")
            return False
            
    def _save_cache(self) -> None:
        """Save relationship data to cache"""
        try:
            cache_path = self._get_cache_path()
            with open(cache_path, 'wb') as f:
                pickle.dump(self.d, f)
        except Exception as e:
            print(f"Error saving cache: {e}")

    def get_company_info(self, bbg_cid: str) -> Optional[CompanyInfo]:
        """Get company information by Bloomberg ID"""
        try:
            ticker = self.bbg_cid_to_ticker(bbg_cid)
            if not ticker:
                return None
                
            return CompanyInfo(
                sector=self.d.get("sector_map", {}).get(bbg_cid, "Unknown"),
                ranking=0.0,  # Will be set by customer/supplier specific methods
                ticker=ticker,
                refinitive_id=self.d["bbg cid to refinitiv id map"].get(bbg_cid, ""),
                name=self.d["bbg cid to name map"].get(bbg_cid, "")
            )
        except Exception as e:
            print(f"Error getting company info for {bbg_cid}: {e}")
            return None

    def get_customers_by_ticker(self, ticker: str) -> Dict[str, RelationshipData]:
        """Get customer relationships for a company"""
        try:
            bbg_cid = self.ticker_to_bbg_cid(ticker)
            if not bbg_cid:
                return {}
                
            result = {}
            customer_data = self.d["top customers final struct"].get(bbg_cid, {})
            
            for cust_id, data in customer_data.items():
                company_info = self.get_company_info(cust_id)
                if not company_info:
                    continue
                    
                company_info.ranking = data.get("ranking", 0.0)
                consensus = self._get_consensus_data(cust_id)
                
                result[cust_id] = RelationshipData(
                    company_info=company_info,
                    consensus_data=consensus,
                    ect_analysis=None  # Will be filled by ECTAnalyzer
                )
                
            return result
        except Exception as e:
            print(f"Error getting customers for {ticker}: {e}")
            return {}

    def get_suppliers_by_ticker(self, ticker: str) -> Dict[str, RelationshipData]:
        """Get supplier relationships for a company"""
        try:
            bbg_cid = self.ticker_to_bbg_cid(ticker)
            if not bbg_cid:
                return {}
                
            result = {}
            supplier_data = self.d["top suppliers final struct"].get(bbg_cid, {})
            
            for supp_id, data in supplier_data.items():
                company_info = self.get_company_info(supp_id)
                if not company_info:
                    continue
                    
                company_info.ranking = data.get("ranking", 0.0)
                consensus = self._get_consensus_data(supp_id)
                
                result[supp_id] = RelationshipData(
                    company_info=company_info,
                    consensus_data=consensus,
                    ect_analysis=None  # Will be filled by ECTAnalyzer
                )
                
            return result
        except Exception as e:
            print(f"Error getting suppliers for {ticker}: {e}")
            return {}

    def _get_consensus_data(self, bbg_cid: str) -> Optional[ConsensusData]:
        """Get consensus data for a company"""
        try:
            cons_data = self.d["cons data final struct"].get(bbg_cid, {})
            if not cons_data:
                return None
                
            return ConsensusData(
                quarterly_growth=cons_data.get("quarterly_growth", []),
                period=f"Q{self.quarter} {self.year}"
            )
        except Exception as e:
            print(f"Error getting consensus data for {bbg_cid}: {e}")
            return None

    # ID conversion utilities
    def ticker_to_bbg_cid(self, ticker: str) -> Optional[str]:
        return self.d["ticker to bbg cid map"].get(ticker)

    def bbg_cid_to_ticker(self, bbg_cid: str) -> Optional[str]:
        reverse_map = {v: k for k, v in self.d["ticker to bbg cid map"].items()}
        return reverse_map.get(bbg_cid)

    def ticker_to_refinitive_id(self, ticker: str) -> Optional[str]:
        bbg_cid = self.ticker_to_bbg_cid(ticker)
        if not bbg_cid:
            return None
        return self.d["bbg cid to refinitiv id map"].get(bbg_cid)